In [9]:
import yaml
import os
from glob import glob
import os
import pandas as pd
import json
import pickle
import numpy as np

# Choose Folder Here


In [5]:
#dir = '/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2'
#dir = '/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2'
dir = '/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1' #'/home/gsantiago/summer_bee_data/open_sets/new_open_max_ids_batch1' # '/home/gsantiago/summer_bee_data/closed_sets_8_ids_monocolor_batch2'
#dir to start working with 

#make new wandb project based on dir name


test_file = '/home/gsantiago/summer_bee_data/open_sets/open_reference_query_testing_batch2/summer_bee_dataset_closed_test_bee_query_64_ids_batch2.csv' # '/home/gsantiago/summer_bee_data/closed_test_batch1/summer_bee_dataset_closed_test_bee_sample_num_None.csv'
#test on all of batch 2 (open set) gonna also test on all of batch 1 

valid_file = '/home/gsantiago/summer_bee_data/open_sets/open_reference_query_testing_batch2/summer_bee_dataset_closed_test_bee_valid_64_ids_batch2.csv' #/home/gsantiago/summer_bee_data/closed_sets_max_ids_batch2/summer_bee_dataset_closed_train_bee_balanced_sample_num_2.csv'
# run valid on smaller subset of test_set to speed training 

reference_file = '/home/gsantiago/summer_bee_data/closed_sets_max_ids_batch2/summer_bee_dataset_closed_train_bee_balanced_sample_num_2.csv'
#reference knn on smaller subset of batch 1

results_file = '/home/lmeyers/ReID_complete/color_detect_experiments/Color_detect_few_shot_expirament_results_tracking.csv'

file_paths = []

In [6]:
# Get file list 

for root, dirs, files in os.walk(dir):
    files = files
for f in files:
    print(root+r'/'+f)
    train_file = root+r'/'+f
    file_paths.append(train_file)
#     continue

#files.sort()

/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_max.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_16.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_04.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_32.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_08.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_64.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_02.csv


## If setting up two folders to train one after the other

In [ ]:
dir2 = '/home/gsantiago/summer_bee_data/closed_sets_8_ids_monocolor_batch2'
files2 = []
for root2, dirs2, files2 in os.walk(dir2):
    files2 = files2
for f2 in files2:
    #print(root+r'/'+f2)
    train_file2 = root2+r'/'+f2
    file_paths.append(train_file2)
#     continue


print(file_paths)

['/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2/summer_bee_dataset_closed_train_bee_8_ids_double_colors_batch2_sample_num_4.csv', '/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2/summer_bee_dataset_closed_train_bee_8_ids_double_colors_batch2_sample_num_2.csv', '/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2/summer_bee_dataset_closed_train_bee_8_ids_double_colors_batch2_sample_num_8.csv', '/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2/summer_bee_dataset_closed_train_bee_8_ids_double_colors_batch2_sample_num_16.csv', '/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2/summer_bee_dataset_closed_train_bee_8_ids_double_colors_batch2_sample_num_20.csv', '/home/gsantiago/summer_bee_data/closed_sets_8_ids_double_color_batch2/summer_bee_dataset_closed_train_bee_8_ids_double_colors_batch2_sample_num_max.csv', '/home/gsantiago/summer_bee_data/closed_sets_8_ids_monocolor_batch2/summer_be

# Training

Change num epochs to test and any run filters (if statement just above with(open(new_yml_file)))

In [ ]:
epochs_to_test = [400]
for n in epochs_to_test: 
    for i in range(len(file_paths)):
        #train file path
        train_file = file_paths[i]
        #train_file = root+r'/'+files[i]
        wandb_name = 'color_detect_'+ os.path.basename(os.path.dirname(train_file))
        # initilize and make new dir for run
        #run_num = '0'+str(i) #I've decided this makes it harder
        run_str = os.path.basename(train_file)[34:-4]
        run_dir_name = run_str+'/'
        if not os.path.exists(run_dir_name):
            os.mkdir(run_dir_name)

        split_parts = run_str.rsplit('_', 10)
        print(split_parts)
        # Check if there is at least one underscore in the string
        if len(split_parts) > 1:
            # Get the substring after the last underscore
            num_images = split_parts[-1]
            num_ids = split_parts[0]
        else:
            # Handle the case where there are no underscores in the string
            num_images = run_str

        print(num_images)
        print(num_ids)

        #filter particular runs (if needed)
        #if num_images == 'max':
        
        
        #"""#skipping max because I already ran it
        if  num_images not in ['64']:
            print('Skipping')
            continue
        #"""

        ##---------- Initilize new config .yml for new training file---------------

        #open config yaml to update experiment params
        with open('/home/lmeyers/ReID_complete/color_detect_template.yml', 'r') as fo:
            config = yaml.safe_load(fo)
        
        #Update params
        #config['model_settings']['num_labels']= run_str[0]
        #print('Num labels ',run_str[0])

        #Check if batch size needs to be updated
        df = pd.read_csv(train_file)
        if config['data_settings']['batch_size'] > len(df):
            config['data_settings']['batch_size'] = len(df)
            print('Updated batch to contain all Data. Size = ',len(df))

        #Check if print_k needs to be updated for small dataset
        print_k = config['train_settings']['print_k']
        if print_k > len(df)/config['data_settings']['batch_size']:
            print_k = int(np.floor(len(df)/config['data_settings']['batch_size']))
            config['train_settings']['print_k'] = print_k
            print('Updating print_k to contain whole epoch. Num_batches =',print_k)
        
        #Testing a differnt num epochs (EXPIRAMENT HERE)
        num_epochs =  1550
        config['train_settings']['num_epochs'] = num_epochs
        print("Number of Epochs",num_epochs)
        
        #updating datafiles
        config['data_settings']['datafiles']['train']=train_file
        config['data_settings']['datafiles']['reference']= reference_file


        #config['data_settings']['datafiles']['train']=train_csv
        config['data_settings']['datafiles']['test'] = test_file
        config['data_settings']['datafiles']['valid']= valid_file 
        config['data_settings']['datafiles']['query']= test_file

        #update Model path
        config['model_settings']['model_path'] = './'+run_dir_name+run_str+'.pth'

        #update wandb_project_name
        config['train_settings']['wandb_project_name'] = wandb_name
        config['train_settings']['wandb_dir_path'] = '/home/lmeyers/ReID_complete/color_detect_experiments/'+ run_dir_name #this should make a seperate wandb folder for runs

        #save yml
        new_yml_file = './'+run_dir_name+run_str+'.yml'


        with open(new_yml_file, 'w') as fo:
                yaml.dump(config,fo)   

        #---------- actually run training too--------------
        !python /home/lmeyers/ReID_complete/pytorch_train_and_eval_color_detect.py --config_file {new_yml_file}

        # Save model to wandb file location to prevent overwriting
        !cp {config['model_settings']['model_path']} {config['train_settings']['wandb_dir_path']+'/wandb/latest-run/files/'+os.path.basename(config['model_settings']['model_path'])}

        with open('/home/lmeyers/ReID_complete/color_detect_experiments/results.pkl','rb') as fi:
            results = pickle.load(fi)
    
      
        # Write out run summary to results tracking document
        results_df = pd.read_csv(config['eval_settings']['results_file'])
        results_df.loc[len(results_df)] = {'run_str': run_str,
                                            'wandb_id':results['wandb_id'],
                                            'num_ids':num_ids,
                                            'num_images_per_id':num_images,
                                            'total_training_images':len(pd.read_csv(train_file)),
                                            'batch_size':config['data_settings']['batch_size'],
                                            'num_epochs':config['train_settings']['num_epochs'],
                                            'train_loss':results['train_loss'],
                                            'valid_loss':results['valid_loss'],
                                            'percent_correct_samples':results['percent_correct_samples'],
                                            'total_acc':results["Total Acc:"],
                                            'total_precision':results["Total Precision:"],
                                            'total_recall':results["Total Recall:"],
                                            'training_file':train_file,
                                            'reference_file':reference_file,
                                            'query_file':test_file,
                                            'start_time':results['start_time'],
                                            'train_time':results['train_time'],
                                            'stop_epoch':results['stop_epoch']}
        results_df.to_csv(config['eval_settings']['results_file'],index=False)

        

        

        
    
    

['4', 'ids', 'batch1', 'sample', 'num', 'max']
max
4
Skipping
['4', 'ids', 'batch1', 'sample', 'num', '16']
16
4
Skipping
['4', 'ids', 'batch1', 'sample', 'num', '04']
04
4
Skipping
['4', 'ids', 'batch1', 'sample', 'num', '32']
32
4
Skipping
['4', 'ids', 'batch1', 'sample', 'num', '08']
08
4
Skipping
['4', 'ids', 'batch1', 'sample', 'num', '64']
64
4
Updating print_k to contain whole epoch. Num_batches = 4
Number of Epochs 1550
2023-11-19 17:08:40.361833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 17:08:41.545957: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: lqmeyers (meyers_luke_lab). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.0 

In [6]:

# here we can specify what we want to train. If we want everything, remove max, it should work
directory = '/home/gsantiago/summer_bee_data/closed_sets_max*'
closed_set_directory = '/home/gsantiago/summer_bee_data/closed_test_'
closed_test = "/home/gsantiago/summer_bee_data/closed_test_"

model_path = '/home/gsantiago/ReID_model_training/model_path/'
for path in glob(directory):
    
    
    batch_number = path.split('_')[-1]
    
    all_csv_directories = path+'/*'
    full_dir_name = path.split('/')[-1]
#     print(full_dir_name)
#     continue
    type_of_split = full_dir_name.replace('closed_sets', '').replace(batch_number,'')
    #open_set_for_test = open_sets+"open"+type_of_split

    #print(batch_number)
    
    closed_for_this_test= closed_test+  batch_number+'/*'
    
#     if(batch_number == 'batch1'):
        
#         #open_set_for_test = open_set_for_test+'batch2/'+"summer_bee_dataset_open_train_bee_balanced_batch2_sample_num_max.csv"
#     else:
#         #open_set_for_test = open_set_for_test+'batch1/'+"summer_bee_dataset_open_train_bee_balanced_batch1_sample_num_max.csv"
        
#     print(open_set_for_test)
    
    with open('./pytorch_train_and_eval_reid.yml', 'r') as f:
        config = yaml.safe_load(f)
    for train_csv in glob(all_csv_directories):
        #print('train_csv')
        #print(train_csv)


        

        #changing train folder
#         print(closed_for_this_test)
#         print(glob(closed_for_this_test))
        train_df = pd.read_csv(train_csv)
        num_labels = train_df['ID'].nunique()
        #print(num_labels)
        
        #batch setting
        config['data_settings']['batch_size']= 64
        
        #updating labels
        
        config['model_settings']['num_labels']=num_labels
        
        #updating datafiles
        config['data_settings']['datafiles']['train']=train_csv
        config['data_settings']['datafiles']['reference']= train_csv
  

        #config['data_settings']['datafiles']['train']=train_csv
        config['data_settings']['datafiles']['test'] = glob(closed_for_this_test)[0]
        config['data_settings']['datafiles']['valid']= glob(closed_for_this_test)[0]
        config['data_settings']['datafiles']['query']= glob(closed_for_this_test)[0]
        
        model_directory =(model_path+"closed/"+full_dir_name+'/')
        
        if not os.path.exists(model_directory):
            os.makedirs(model_directory)
        config['model_settings']['model_path']= model_directory
        
        config_name = train_csv.split('/')[-1].replace('.csv', '')
        print(config['data_settings']['datafiles'])

        new_yml_file = "./different_yml_configs/closed/"+config_name+'_config.yml'
        output_file = "./all_outputs/"+(new_yml_file.split('/')[-1].replace('_config.yml', '_output.txt'))
       
        with open(new_yml_file, 'w') as f:
            yaml.dump(config,f)
        
        !python pytorch_train_and_eval_reid.py --config_file {new_yml_file} > {output_file} 

    
    
    



FileNotFoundError: [Errno 2] No such file or directory: './pytorch_train_and_eval_reid.yml'

# open set


In [ ]:

# here we can specify what we want to train. If we want everything, remove max, it should work
directory = '/home/gsantiago/summer_bee_data/open_sets/open_max*'
open_set_directory = '/home/gsantiago/summer_bee_data/open_sets/'
open_test = "/home/gsantiago/summer_bee_data/open_sets/"

model_path = '/home/gsantiago/ReID_model_training/model_path/'
for path in glob(directory):
    
    #print(path)
    batch_number = path.split('_')[-1]
    #print(batch_number)
    
    all_csv_directories = path+'/*'
    #print(glob(all_csv_directories))
    full_dir_name = path.split('/')[-1]
    #print(full_dir_name)
    type_of_split = full_dir_name.replace(batch_number,'')
    #print("type_of_split")
    #print(type_of_split)
   

    #print(batch_number)
    
    #open_for_this_test= open_test+  batch_number+'/*'
    
    if(batch_number == 'batch1'):
        
        open_set_for_test = open_test+type_of_split+'batch2/'+"summer_bee_dataset_open_train_bee_balanced_batch2_sample_num_max.csv"
        
    else:
        open_set_for_test = open_test+type_of_split+'batch1/'+"summer_bee_dataset_open_train_bee_balanced_batch1_sample_num_max.csv"
        
    #print(open_set_for_test)
    
    with open('./pytorch_train_and_eval_reid.yml', 'r') as f:
        config = yaml.safe_load(f)
    #print(glob(all_csv_directories))
    for train_csv in glob(all_csv_directories):
        print('train_csv')
        #print(train_csv)


        # optional, skipping  the finished cvs
        
        #changing train folder
        #print("open_set_for_test")
        #print(open_set_for_test)
        #print(glob(open_set_for_test))
        train_df = pd.read_csv(train_csv)
        num_labels = train_df['ID'].nunique()
        print(num_labels)
        
        #batch setting
        config['data_settings']['batch_size']= 64
        
        #updating labels
        
        config['model_settings']['num_labels']=num_labels
        
        #updating datafiles
        config['data_settings']['datafiles']['train']=train_csv
        config['data_settings']['datafiles']['test'] = open_set_for_test
        config['data_settings']['datafiles']['valid']= open_set_for_test
        config['data_settings']['datafiles']['reference']= train_csv
        config['data_settings']['datafiles']['query']= open_set_for_test
        
        
                
        model_directory =(model_path+"open/"+full_dir_name+'/')
        train_model_directory = model_directory+'train/'
        eval_model_directory = model_directory +'eval/'
        
        if not os.path.exists(model_directory):
            os.makedirs(model_directory)
            os.makedirs(train_model_directory)
            os.makedirs(eval_model_directory)
            
        #sample size for model, saved as name
        
        sample_num = train_csv.split('_')[-1].replace(".csv",'_samples.pth')
            
        config['model_settings']['model_path']= train_model_directory+sample_num
        config['eval_settings']['model_path'] = eval_model_directory +sample_num
        

        
        config_name = train_csv.split('/')[-1].replace('.csv', '')
        #print(config['data_settings']['datafiles'])

        new_yml_file = "./different_yml_configs/open/"+config_name+'_config.yml'
        output_file = "./all_outputs/"+(new_yml_file.split('/')[-1].replace('_config.yml', '_output.txt'))
       
        with open(new_yml_file, 'w') as f:
            yaml.dump(config,f)
        
        !python pytorch_train_and_eval_reid.py --config_file {new_yml_file} > {output_file} 

    
    
    



train_csv
64
2023-10-15 22:44:31.603382: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 22:44:32.942356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: gabriel-santiago21 (meyers_luke_lab). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.1
wandb: Run data is saved locally in /home/gsantiago/ReID_model_training/wandb/run-20231015_224436-pfstr74p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run daily-plant-25
wandb: ⭐️ View project at https://wandb.ai/meyers_luke_lab/ReID_Full_dataset_open
wandb: 🚀 View 